In [1]:
import mne
import  numpy as np
import os
import matplotlib.pyplot as plt
from mne.preprocessing import ICA

import warnings

# Ignore all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import TwoSlopeNorm

import mne
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.time_frequency import tfr_multitaper

In [2]:



eog = ['AF1','AF2']


misc = ['thumb_near',
 'thumb_far',
 'thumb_index',
 'index_near',
 'index_far',
 'index_middle',
 'middle_near',
 'middle_far',
 'middle_ring',
 'ring_near',
 'ring_far',
 'ring_little',
 'litte_near',
 'litte_far',
 'thumb_palm',
 'wrist_bend',
 'roll',
 'pitch',
 'gesture',
 'armeodummy',
 'handPosX',
 'handPosY',
 'handPosZ',
 'elbowPosX',
 'elbowPosY',
 'elbowPosZ',
 'ShoulderAdductio',
 'ShoulderFlexionE',
 'ShoulderRotation',
 'Elbow',
 'ProSupination',
 'Wrist',
 'GripPressure'
 ]
scalings = dict(mag=1e-12, grad=4e-11, eeg=10e-5, eog=150e-6, ecg=5e-4,
     emg=1e-3, ref_meg=1e-12, misc=1e-3, stim=1,
     resp=1, chpi=1e-4, whitened=1e2)


In [3]:
import os

def get_eeg_sessions(base_path, subject_name):
    """

    """

    # Subject path
    path = os.path.join(base_path, subject_name)
    
    # Imagery folder path
    path_2 = os.path.join(path, "Imagery")
    
    # List files in Imagery folder
    files = os.listdir(path_2)

    # Filter .vhdr files
    eeg_files = [f for f in files if f.endswith(".vhdr")]

    session_1 = []
    session_2 = []

    for file in eeg_files:
        # Session 1: contains 1,2,3,4
        if any(x in file for x in ["1", "2", "3", "4"]):
            session_1.append(file)

        # Session 2: contains 5,6,7,8
        if any(x in file for x in ["5", "6", "7", "8"]):
            session_2.append(file)

    return session_1, session_2


In [4]:

def eeg_raw(path_2,eeg_file):
       
    raws_2 = []
    raws = [mne.io.read_raw_brainvision(path_2 +f,eog=eog, preload = True) for f in eeg_file]
    try:
        for i in range(len(raws)):
            Temp= list(set(raws[i].pick(["eeg"]).info["ch_names"])-set(["F3","C3","P3","F4","C4","P4","Fz","Cz","Pz","TP9"]))
            raws_2.append(raws[i].pick(["eeg"]).drop_channels(Temp))
#             print(raws[i].pick(["eeg"]).info["ch_names"])

        raws_2 = mne.io.concatenate_raws(raws_2)
        raws_2.set_eeg_reference(ref_channels=['TP9'])
        raws_2.drop_channels(['TP9'])
    except:
        raws_2 = []
        for i in range(len(raws)):
            Temp= list(set(raws[i].pick(["eeg"]).info["ch_names"])-set(["F3","C3","P3","F4","C4","P4","Fz","Cz","Pz"]))
            raws_2.append(raws[i].pick(["eeg"]).drop_channels(Temp))
#             print(raws[i].pick(["eeg"]).info["ch_names"])

        raws_2 = mne.io.concatenate_raws(raws_2)
        
        
    
    return raws_2


In [5]:
def event_name_id(raws):
    events, event_dict = mne.events_from_annotations(raws)
    print(event_dict,events)
    annotations_dict = {k:k[8:] if k[:8] == 'Comment/' else k for k in event_dict.keys()}
    print(annotations_dict)
    event_id = {"Close":event_dict["Comment/Close"],
        "Open":event_dict["Comment/Open"],
        "Rest":event_dict["Comment/Rest"]}
    print(event_id)
    
    return events,event_id

In [6]:
def epoching(raw,events,event_id):
    tmin =  0 # start of each epoch (in sec)
    tmax =  3  # end of each epoch (in sec)
    # baseline = (None, 0)
    # baseline =  (0.1,0.2)
    baseline = None #(-1,0)
    
    
    
#     sample_freq = 250
#     raw.resample(sample_freq)
    
    
    
    # Create epochs
    epochs = mne.Epochs(raw,
                        events, event_id,
                        tmin, tmax,
                        baseline=baseline,
                        preload=True
                       )
    return epochs

In [7]:
def event_data(epoch):
    
    event_len = len(epoch["Open"].get_data())
    Close_data = epoch["Close"][:event_len].get_data()
    Open_data = epoch["Open"].get_data()
    Rest_data = epoch["Rest"][:event_len].get_data()
    
    print(Close_data.shape , Open_data.shape , Rest_data.shape)
    return Close_data, Open_data, Rest_data

In [14]:
def data_file_save(path_2,eeg_file,subject):
    x = eeg_raw(path_2,eeg_file)
    y, z = event_name_id(x)
    a = epoching(x,y,z)
    c, o, r = event_data(a)
    label_c =  np.zeros(len(c))+1
    label_o = np.zeros(len(c))+2
    label_r = np.zeros(len(r))+3
    final_label = np.concatenate((label_c,label_o,label_r ))
    np.save(f"./Session_Data_file/label_{subject}.npy",final_label,)
    data = np.concatenate((c, o, r))
    np.save(f"./Session_Data_file/data_{subject}.npy", data)

In [15]:
base_path = "./Subject_dependent/"

In [12]:

def save_subject_sessions(d, idx, base_path= base_path):
    
    # Get subject name from list
    subject_name = d[idx]

    # Create output folder
    os.makedirs("/Session_Data_file/", exist_ok=True)

    # Get session files
    session_1, session_2 = get_eeg_sessions(base_path, subject_name)

    # Imagery folder path
    path_2 = os.path.join(base_path, subject_name, "Imagery/")

    # Save session-wise data
    data_file_save(path_2, session_1, subject_name + "_1")
    data_file_save(path_2, session_2, subject_name + "_2")

    print(f"Saved data for subject: {subject_name}")


In [16]:
d = os.listdir(base_path)

for i,j in enumerate(d):
    print(i,j)

0 Abha
1 Aman
2 Anuj Sir
3 Aryan
4 Dipti
5 Harsha
6 jay
7 Mayank
8 Mintu
9 Moumita
10 Mudavath
11 Prashant
12 Ritesh
13 Sharon
14 Shivam
15 Soinik
16 Suma
17 Zeeshan


In [17]:
%%capture

for i in range(len(d)):
        save_subject_sessions(d, i)
  